# Exercise 2 - Getting Started with Experiments

Azure Machine Learning (Azure ML) is a cloud-based service that enables data scientists, AI software engineers, and others to collaborate on machine learning projects and manage data science workloads at scale. One of the most fundamentals tasks that data scientists need to perform is to create and run experiments that process and analyze data. In this exercise, you'll learn how to use an Azure ML *experiment* to run Python code and record values extracted from data.

## Task 1: Install the Azure ML SDK for Python

The Azure ML SDK for Python provides classes you can use to work with Azure ML in your Azure subscription. The SDK is pre-installed in the Azure ML notebooks environment, but it's worth checking that you have the latest version of the package installed. Run the cell below to install the **azureml-sdk** Python package, including the optional *notebooks* component; which provides some functionality specific to the Jupyter Notebooks environment.

> **More Information**: For more details about installing the Azure ML SDK and its optional components, see the [Azure ML SDK Documentation](https://docs.microsoft.com/en-us/python/api/overview/azure/ml/install?view=azure-ml-py)

In [ ]:
!pip install --upgrade azureml-sdk[notebooks]

import azureml.core
print("Ready to use Azure ML", azureml.core.VERSION)

## Task 2: Sign Into your Azure Subscription

Now that you've installed the SDK, you can use it to create, manage, and use Azure ML related objects in your Azure subscription, which means you'll need an authenticated connection between the code in this notebook and your Azure subscription. To create this authenticated connection, you can use the **authentication** module in the Azure ML SDK. In this case, you'll use the **InteractiveLoginAuthentication** class to generate a session token.

Run the cell below, and when prompted, click the https://microsoft.com/devicelogin link and enter the automatically generated code. Then, sign into your Azure subscription in the browser tab that is opened. After you have successfully signed in, you can close the browser tab that was opened and return to this notebook.

In [ ]:
from azureml.core import authentication

auth = authentication.InteractiveLoginAuthentication()
print('Signed in')

## Task 3: Connect to a Workspace

All experiments and associated resources are managed within an Azure ML *workspace*. You can connect to an existing workspace, or create a new one using the Azure ML SDK.

In the code below, enter appropriate values for the *SUBSCRIPTION_ID*, *RESOURCE_GROUP*, *WORKSPACE_NAME*, and *REGION* constants for your workspace (if you have already created a workspace, you can find all of these details on its **Overview** page in the [Azure portal](https://portal.azure.com)). Then run the cell to create your workspace.

> **Note**: If you hadn't previously created an authenticated session, you'd automatically be prompted to sign into your Azure subscription!

In [ ]:
from azureml.core import Workspace

SUBSCRIPTION_ID = '<YOUR_AZURE_SUBSCRIPTION_ID>' # Get this from the Azure portal
RESOURCE_GROUP_NAME  = 'aml-resource-group' # Or any resource group name of your choice - if it doesn't exist, it will be created
WORKSPACE_NAME  = 'aml-workspace' # Or a name of your choice - if it doesn't exist, it will be created
REGION = 'eastus2'# Or a region of your choice

ws = None
try:
    # Find existing workspace
    ws = Workspace(workspace_name=WORKSPACE_NAME,
                   subscription_id=SUBSCRIPTION_ID,
                   resource_group= RESOURCE_GROUP_NAME)
    print (ws.name, "found.")
except Exception as ex:
    # If workspace not found, create it
    print(ex.message)
    print("Attempting to create new workspace...")
    ws = Workspace.create(name=WORKSPACE_NAME, 
                      subscription_id=SUBSCRIPTION_ID,
                      resource_group=RESOURCE_GROUP_NAME,
                      create_resource_group=True,
                      location=REGION 
                     )
    print(ws.name, "created.")
finally:
    # Save the workspace configuration for later
    if ws != None:
        ws.write_config()
        print(ws.name, "saved.")

In the [Azure ML Web Interface](https://ml.azure.com), note that you can manage various Azure ML assets, such as *experiments*, *pipelines*, *compute*, *models*, and others. You will explore these kinds of asset in subsequent exercises.

In the code above, note that you used the **write_config** method to save the workspace configuration. This saved a JSON configuration file in a hidden folder named **.azureml**, which you can verify with the following cell.

In [ ]:
import json
# Print the config.json file
with open("./.azureml/config.json","r") as f:
    json_data = json.loads(f.read())
    print(json.dumps(json_data, indent=2))

This saved configuration file enables you to easily obtain a reference to the workspace by simply loading it, as demonstrated in the following cell. Note that this method will prompt you to reauthenticate against your Azure subscription if your session has expired.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, "loaded")

## Task 4: Run an Experiment

So far, you've spent a lot of time setting up your Azure ML workspace; and you may be beginning to wonder what benefits this will bring to your day-to-day data science activities. Well, there are lots of benefits, which we'll explore in detail in later exercises; but for now, let's see how Azure ML can help track metrics from a simple experiment that uses Python code to examine some data.

In this case, you'll use a simple dataset that contains details of patients that have been tested for diabetes. You'll run an experiment to explore the data, extracting statistics, visualizations, and data samples. Most of the code you'll use is fairly generic Python, such as you might run in any data exploration process. However, with the addition of a few lines, the code uses an Azure ML *experiment* to log details of the run.

In [ ]:
from azureml.core import Experiment, Run
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace = ws, name = "diabetes-experiment")
print("Starting experiment:", experiment.name)

# Start logging data from the experiment
run = experiment.start_logging()

# load the diabetes dataset
data = pd.read_csv('data/diabetes.csv')

# Count the rows and log the result
row_count = (len(data))
run.log("observations", row_count)

# Create box plots for each feature variable by the "diabetic" label and log them
num_cols = data.columns[:-1]
for col in num_cols:
    fig = plt.figure(figsize=(9, 6))
    ax = fig.gca()
    data.boxplot(column = col, by = "Diabetic", ax = ax)
    ax.set_title(col + ' by Diabetic')
    ax.set_ylabel(col)
    run.log_image(name = col, plot = fig)
plt.show()

# Create a list of mean diabetes pedigree per age and log it
mean_by_age = data[["Age", "DiabetesPedigree"]].groupby(["Age"]).mean().reset_index()
ages = mean_by_age["Age"].tolist()
pedigrees = mean_by_age["DiabetesPedigree"].tolist()
for index in range(len(ages)):
       run.log_row("Mean Diabetes Pedigree by Age", Age = ages[index],Diabetes_Pedigree = pedigrees[index])

# Save a sample of the data and upload it to the experiment output
data.sample(100).to_csv("sample.csv", index=False, header=True)
run.upload_file(name = 'outputs/sample.csv', path_or_stream = './sample.csv')

# Complete tracking and get link to details
run.complete()


## Task 5: View Experiment Results

After the experiment has been finished, you can use the **run** object to get information about the run and its outputs:

In [ ]:
# Get run details
details = run.get_details()
print(details)

# Get logged metrics
metrics = run.get_metrics()
print(json.dumps(metrics, indent=2))

# Get output files
files = run.get_file_names()
print(json.dumps(files, indent=2))


You can also view details of experiment runs in the [Azure ML web interface](https://ml.azure.com). Go there now and explore the details for this run on the **Experiments** tab. When you view the details for the run, you can see metrics that were logged, the images that were created by plots, and output values that were generated.

## Task 6: Run an Experiment Script

In the previous example, you ran an experiment inline in this notebook. A more flexible solution is to create a separate script for the experiment, and store it in a folder along with any other files it needs, and then use Azure ML to run the experiment based on the script in the folder.

Let's create a folder for the experiment:

In [ ]:
import os, shutil

# Create an experiment
experiment_name = 'diabetes-experiment'

# Create a folder for the experiment files
experiment_folder = './' + experiment_name
os.makedirs(experiment_folder, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/diabetes.csv', os.path.join(experiment_name, "diabetes.csv"))

Now we'll create a Python script containing the experiment code. Note that this code is the same as the inline code used before, except we use the `Run.get_context()` method to retrieve the experiment run context when the script is run, and we load the diabetes data from the folder where the script is located:

In [ ]:
%%writefile $experiment_folder/diabetes_experiment.py
from azureml.core import Run
import pandas as pd
import matplotlib.pyplot as plt

# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
data = pd.read_csv('./diabetes.csv')

# Count the rows and log the result
row_count = (len(data))
run.log("observations", row_count)

# Create box plots for each feature variable by the "diabetic" label and log them
num_cols = data.columns[:-1]
for col in num_cols:
    fig = plt.figure(figsize=(9, 6))
    ax = fig.gca()
    data.boxplot(column = col, by = "Diabetic", ax = ax)
    ax.set_title(col + ' by Diabetic')
    ax.set_ylabel(col)
    run.log_image(name = col, plot = fig)
plt.show()

# Create a list of mean diabetes pedigree per age and log it
mean_by_age = data[["Age", "DiabetesPedigree"]].groupby(["Age"]).mean().reset_index()
ages = mean_by_age["Age"].tolist()
pedigrees = mean_by_age["DiabetesPedigree"].tolist()
for index in range(len(ages)):
       run.log_row("Mean Diabetes Pedigree by Age", Age = ages[index],Diabetes_Pedigree = pedigrees[index])

# Save a sample of the data and upload it to the experiment output
data.sample(100).to_csv("sample.csv", index=False, header=True)
run.upload_file(name = 'outputs/sample.csv', path_or_stream = './sample.csv')

# Complete tracking and get link to details
run.complete()

Now you're almost ready to run the experiment. There are just a few configuration issues you need to deal with:

1. Create a *Run Configuration* that defines the Python code execution environment - in this case, you'll create a new Conda environment that includes the packages required by the experiment script.
2. Create a *Script Configuration* that identifies the Python script to be run in the experiment, and the environment in which to run it.

The following cell sets up these configuration objects, and then submits the experiment.

In [ ]:
import os
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Experiment, RunConfiguration
from azureml.core import ScriptRunConfig
from azureml.core import Experiment

# create a new RunConfig object
run_config = RunConfiguration()

# Create a Python environment for the experiment
diabetes_env = Environment("diabetes-experiment-env")
diabetes_env.python.user_managed_dependencies = False # we're going to create an environment
diabetes_env.docker.enabled = False # Don't use a docker container (default is true)

# Create a set of package dependencies (conda or pip as required)
diabetes_conda = CondaDependencies.create(conda_packages=['pandas','ipykernel','matplotlib'],
                                          pip_packages=['azureml-sdk','argparse','pyarrow'])

# Add the dependencies to the environment
diabetes_env.python.conda_dependencies = diabetes_conda

# Use this python environment in the run config
run_config.environment = diabetes_env

# Create a script config
src = ScriptRunConfig(source_directory=experiment_folder, 
                      script='diabetes_experiment.py',
                      run_config=run_config) 

# submit the experiment
experiment = Experiment(workspace = ws, name = experiment_name)
run = experiment.submit(config=src)
run.wait_for_completion(show_output=True)

As before, you can use the [Azure ML web interface](https://ml.azure.com) to view the outputs generated by the experiment, and you can also write code to retrieve the metrics and files it generated:

In [ ]:
# Get logged metrics
metrics = run.get_metrics()
print(json.dumps(metrics, indent=2))

# Get output files
files = run.get_file_names()
print(json.dumps(files, indent=2))

> **More Information**: To find out more about running experiments, see [this topic](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-runs) in the Azure ML documentation. For details of how to log metrics in a run, see [this topic](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-track-experiments).